## 方針
1. 雛形の作成：wikiから現在のチャンプの種類、スキルについて取得
2. 公式ページから過去分を取得
3. 取得した2のデータ構造化しつつまとめる
4. 可視化する→理想はキャラごとの変遷が見れる形

## 実装テスト

In [2]:
##環境準備
import time ## アクセス時間の待機
import requests ## htmlの取得
from bs4 import BeautifulSoup ## htmlの解析
import pandas as pd ##データフレーム関連の使用
import random ## ランダムな数値の生成
import re ## 正規表現
import pickle ## pickleファイルの使用

## 9.1から12.18までの階層構造に対応したプログラム

In [31]:
# 変数準備
data = []
exsist_patch = []

# パッチバージョンをループ
for patch_i in range(9, 13):
    for patch_j in range(1, 28):
        patch_version = f'patch{patch_i}.{patch_j}' # パッチのバージョンを作成
        url = f"https://www.leagueoflegends.com/ja-jp/news/game-updates/patch-{patch_i}-{patch_j}-notes/"
        try: ## ないパッチを避ける
            response = requests.get(url)
            response.raise_for_status()
            soup = BeautifulSoup(response.content, 'html.parser')
            patch_blocks = soup.find_all('div', class_='patch-change-block white-stone accent-before')
            exsist_patch.append([patch_i,patch_j])
            # 変更カラムから抽出
            for block in patch_blocks:
                try:
                    change_title = block.find(class_='change-title').get_text(strip=True)
                    change_details = block.find_all('h4', class_=re.compile(r'change-detail-title|change-detail-title ability-title'))
                    for change_detail in change_details:
                            change_section = change_detail.get_text(strip=True)
                            change_docs = change_detail.find_next_siblings('div', class_='attribute-change')
                            for text in change_docs:
                                change_item = text.find('span', class_='attribute').get_text(strip=True)
                                before_change = text.find('span', class_='attribute-before').get_text(strip=True)
                                after_change = text.find('span', class_='attribute-after').get_text(strip=True)
                                data.append([patch_version, change_title, change_section, change_item,before_change, after_change])
                except AttributeError:
                    continue
            print(f"{patch_version}の取得が完了しました。")
        except requests.exceptions.RequestException as e: # エラーが発生した場合はパス
            print(f"{patch_version}の取得は失敗しました。　理由: {e}")
            continue
        finally:
            time.sleep(random.uniform(3,4))


# 後処理
columns = ['パッチバージョン', '変更対象', '変更概要', '変更部分','変更前', '変更後']
df = pd.DataFrame(data, columns=columns)
f = open('ptach_list.txt', 'wb')
pickle.dump(exsist_patch,f)
df.to_csv("./all_patch_note.csv",index=False)
print('処理が完了しました')



patch9.1の取得が完了しました。
patch9.2の取得が完了しました。
patch9.3の取得が完了しました。
patch9.4の取得は失敗しました。　理由: 404 Client Error: Not Found for url: https://www.leagueoflegends.com/ja-jp/news/game-updates/patch-9-4-notes/
patch9.5の取得は失敗しました。　理由: 404 Client Error: Not Found for url: https://www.leagueoflegends.com/ja-jp/news/game-updates/patch-9-5-notes/
patch9.6の取得は失敗しました。　理由: 404 Client Error: Not Found for url: https://www.leagueoflegends.com/ja-jp/news/game-updates/patch-9-6-notes/
patch9.7の取得は失敗しました。　理由: 404 Client Error: Not Found for url: https://www.leagueoflegends.com/ja-jp/news/game-updates/patch-9-7-notes/
patch9.8の取得は失敗しました。　理由: 404 Client Error: Not Found for url: https://www.leagueoflegends.com/ja-jp/news/game-updates/patch-9-8-notes/
patch9.9の取得は失敗しました。　理由: 404 Client Error: Not Found for url: https://www.leagueoflegends.com/ja-jp/news/game-updates/patch-9-9-notes/
patch9.10の取得は失敗しました。　理由: 404 Client Error: Not Found for url: https://www.leagueoflegends.com/ja-jp/news/game-updates/patch-9-10-notes/


## パッチ12.19~パッチ14.18までに対応したプログラム

In [36]:
# 変数準備
data = []
f = open("./ptach_list.txt", "rb")
exsist_patch = pickle.load(f)
n_i, n_j = 12,18
old_df = pd.read_csv('./all_patch_note.csv')
current_i = int(input("現在のシーズンを半角数字で入力してください"))
# パッチバージョンをループ
for patch_i in range(9, 15):
    for patch_j in range(1, 28):
        if n_i < patch_i or (n_i == patch_i and n_j < patch_j):  # 今のパッチより古かったら実行しない
            patch_version = f'patch{patch_i}.{patch_j}'  # パッチのバージョンを作成
            url = f"https://www.leagueoflegends.com/ja-jp/news/game-updates/patch-{patch_i}-{patch_j}-notes/"
            try:  # ないパッチを避ける
                response = requests.get(url)
                response.raise_for_status()
                soup = BeautifulSoup(response.content, 'html.parser')
                patch_blocks = soup.find_all('div', class_='patch-change-block')
                exsist_patch.append([patch_i,patch_j])
                # 各ブロックから変更を抽出
                for block in patch_blocks:  # block ループを追加
                    change_title_tag = block.find("h3", class_="change-title")
                    if change_title_tag:
                        change_title = change_title_tag.get_text(strip=True)

                    # 各変更セクションの解析
                    for section in block.find_all("h4", class_="change-detail-title"):
                        change_section = section.get_text(strip=True)

                        # section の次に ul 要素が存在するか確認
                        next_siblings = section.find_next_siblings("ul", limit=1)
                        if next_siblings:  # ul 要素が存在する場合
                            for item in next_siblings[0].find_all("li"):
                                # strong タグのテキストを抽出
                                strong_tag = item.find("strong")
                                change_item = strong_tag.get_text(strip=True) if strong_tag else None

                                # 項目のテキスト全体を抽出
                                item_text = item.get_text(strip=True)

                                if '⇒' in item_text:
                                    # 「⇒」がある場合、前半と後半に分ける
                                    parts = item_text.split("⇒", 1)  # 最初の「⇒」で分割
                                    before_change = parts[0].strip()
                                    after_change = parts[1].strip()

                                    # strong タグの内容を前半から除去して重複を避ける
                                    if change_item and change_item in before_change:
                                        before_change = before_change.replace(change_item, '').strip()

                                else:
                                    # 「⇒」がない場合、すべてをafter_changeに入れる
                                    before_change = None
                                    after_change = item_text.strip()

                                # データを追加
                                data.append([patch_version, change_title, change_section, change_item, before_change, after_change])

                print(f"{patch_version}の取得を完了しました。")
            except requests.exceptions.RequestException as e:  # エラーが発生した場合はパス
                print(f"{patch_version}の取得は失敗しました。 理由: {e}")
                continue
            finally:
                time.sleep(random.uniform(3, 4))
        elif patch_i == 14 and patch_j == 18:
            break
    if patch_i == 14 and patch_j == 18:
        break
# 後処理
columns = ['パッチバージョン', '変更対象', '変更概要', '変更部分', '変更前', '変更後']
new_df = pd.DataFrame(data, columns=columns)
save_df = pd.concat([old_df, new_df])
exsist_patch = patch_i,patch_j
f = open('ptach_list.txt', 'wb')
pickle.dump(exsist_patch,f)
save_df.to_csv("./all_patch_note.csv", index=False)
print('処理が完了しました')

patch12.19の取得を完了しました。
patch12.20の取得を完了しました。
patch12.21の取得を完了しました。
patch12.22の取得を完了しました。
patch12.23の取得を完了しました。
patch12.24の取得は失敗しました。 理由: 404 Client Error: Not Found for url: https://www.leagueoflegends.com/ja-jp/news/game-updates/patch-12-24-notes/
patch12.25の取得は失敗しました。 理由: 404 Client Error: Not Found for url: https://www.leagueoflegends.com/ja-jp/news/game-updates/patch-12-25-notes/
patch12.26の取得は失敗しました。 理由: 404 Client Error: Not Found for url: https://www.leagueoflegends.com/ja-jp/news/game-updates/patch-12-26-notes/
patch12.27の取得は失敗しました。 理由: 404 Client Error: Not Found for url: https://www.leagueoflegends.com/ja-jp/news/game-updates/patch-12-27-notes/
patch13.1の取得を完了しました。
patch13.2の取得は失敗しました。 理由: 404 Client Error: Not Found for url: https://www.leagueoflegends.com/ja-jp/news/game-updates/patch-13-2-notes/
patch13.3の取得を完了しました。
patch13.4の取得を完了しました。
patch13.5の取得を完了しました。
patch13.6の取得を完了しました。
patch13.7の取得を完了しました。
patch13.8の取得を完了しました。
patch13.9の取得を完了しました。
patch13.10の取得を完了しました。
patch13.11の取

## 14.18以降

In [163]:

# 変数準備
data = []
f = open("./ptach_list.txt","rb")
exsist_patch = pickle.load(f)
n_i , n_j = 14,18 #exsist_patch[-1]
old_df = pd.read_csv('./all_patch_note.csv')
current_i = int(input("現在のシーズンを半角数字で入力してください"))
# パッチバージョンをループ
for patch_i in range(9, current_i+1):
    for patch_j in range(1, 28):
        if n_i < patch_i  or (n_i == patch_i and n_j < patch_j):#今のパッチより古かったら実行しない
            patch_version = f'patch{patch_i}.{patch_j}' # パッチのバージョンを作成
            url = f"https://www.leagueoflegends.com/ja-jp/news/game-updates/patch-{patch_i}-{patch_j}-notes/"
            try: ## ないパッチを避ける
                response = requests.get(url)
                response.raise_for_status()
                soup = BeautifulSoup(response.content, 'html.parser')
                patch_blocks = soup.find_all('div', class_='patch-change-block white-stone accent-before')
                exsist_patch.append([patch_i,patch_j])
                for patch_block in patch_blocks:
                    try :
                        change_title = patch_block.find("h3", class_="change-title").get_text(strip=True)
                    except:
                        continue
                    try :
                        change_section =  patch_block.find("h4", class_="change-detail-title").get_text(strip=True)
                    except:
                        continue
                    changes = patch_block.find_all('li')
                    for item in changes:
                        # strong タグのテキストを抽出
                        change_item = item.find("strong").get_text(strip=True) if item.find("strong") else None
                        item_text = item.get_text(strip=True)
                        try :
                            if '⇒' in item_text:
                                # 「⇒」がある場合、前半と後半に分ける
                                before_change, after_change = item_text.split("⇒")
                                # 重複を取り除く: strong タグのテキストを before_change から削除
                                if change_item and change_item in before_change:
                                    before_change = before_change.replace(change_item, '').strip()
                                before_change = before_change.strip()
                                after_change = after_change.strip()
                        except:
                            # 「⇒」がない場合、すべてをafter_changeに入れる
                            before_change = None
                            after_change = item_text
                        # データをリストに追加
                        data.append([patch_version, change_title, change_section, change_item, before_change, after_change])
                print(f"{patch_version}の取得を完了しました。")
            except requests.exceptions.RequestException as e: # エラーが発生した場合はパス
                print(f"{patch_version}の取得は失敗しました。　理由: {e}")
                continue
            finally:
                time.sleep(random.uniform(3,4))

#後処理
columns = ['パッチバージョン', '変更対象', '変更概要', '変更部分', '変更前', '変更後']
new_df = pd.DataFrame(data, columns=columns)
save_df = pd.concat([old_df, new_df])
exsist_patch = patch_i,patch_j
f = open('ptach_list.txt', 'wb')
pickle.dump(exsist_patch,f)
save_df.to_csv("./all_patch_note.csv", index=False)
print('処理が完了しました')

patch14.19の取得を完了しました。
patch14.20の取得を完了しました。
patch14.21の取得を完了しました。
patch14.22の取得は失敗しました。　理由: 404 Client Error: Not Found for url: https://www.leagueoflegends.com/ja-jp/news/game-updates/patch-14-22-notes/
patch14.23の取得は失敗しました。　理由: 404 Client Error: Not Found for url: https://www.leagueoflegends.com/ja-jp/news/game-updates/patch-14-23-notes/
patch14.24の取得は失敗しました。　理由: 404 Client Error: Not Found for url: https://www.leagueoflegends.com/ja-jp/news/game-updates/patch-14-24-notes/
patch14.25の取得は失敗しました。　理由: 404 Client Error: Not Found for url: https://www.leagueoflegends.com/ja-jp/news/game-updates/patch-14-25-notes/
patch14.26の取得は失敗しました。　理由: 404 Client Error: Not Found for url: https://www.leagueoflegends.com/ja-jp/news/game-updates/patch-14-26-notes/
patch14.27の取得は失敗しました。　理由: 404 Client Error: Not Found for url: https://www.leagueoflegends.com/ja-jp/news/game-updates/patch-14-27-notes/
